미니 프로젝트 목차
    - ① 데이터 연결하기
    - ② ARIMA 실습해보기!
    - ③ 데이터 확인하기
    - ④ 분산을 일정하게 만들기 - log transformation
    - ⑤ 차분을 통해 분석하기
    - ⑥ Auto ARIMA 적용 및 결과 확인
    - ⑦ ARCH 모델 실습해보기

- ① 데이터 연결하기
    - 데이터가 있는 폴더를 연결합니다.
    - arch를 설치합니다.
- ② ARIMA 실습해보기
    - SARIMA를 통해 예측
- ③ 데이터 확인하기

In [ ]:
!pip install arch==6.3.0

In [ ]:
pip install pmdarima

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # acf plot 및 pacf plot을 그리기 위한 라이브러리
from statsmodels.tsa.arima_model import ARIMA # ARIMA 모델
import pmdarima as pm # Auto ARIMA 모델

In [ ]:
# air passengers 데이터 불러오기
ap = pd.read_csv('/content/AirPassengers.csv')
# 'Month' column을 삭제(필요없는 컬럼)
ap.drop('Month', axis = 1 ,inplace = True)

In [ ]:
ap

In [ ]:
plt.plot(ap)
plt.show()

- ④ 분산을 일정하게 만들기 - log transformation
    - 로그변환(Log Transformation)을 통해 분산을 일정하게 조절합니다.
    - 분산이 일정하게 적용되었는지 시각화한 후 확인합니다.

In [ ]:
#log transformation
ap_transformed = np.log(ap)
ap_transformed

In [ ]:
plot_acf(ap_transformed) # acf plot, 점차 감소하는 모양새를 지니고 있음
plt.plot()

In [ ]:
plot_pacf(ap_transformed) # pacf plot, lag 2까지 유의미한 결과를 보임
plt.plot();

- ⑤ 차분을 통해 분석하기
    - ACF, PACF Plot과 차분을 활용하여 분석합니다.

In [ ]:
ap_diff = ap_transformed.diff()
ap_diff = ap_diff.dropna()

In [ ]:
# lag 1과 lag 12에서 유의미한 lag가 포착됨
plot_acf(ap_diff)
plt.plot();

In [ ]:
# lag 1, lag 8~12쯤에서 유의미한 lag가 포착됨
plot_pacf(ap_diff)
plt.plot();

In [ ]:
ap_transformed

In [ ]:
# test 부분 분리
train_size = int(len(ap_transformed)*0.8)

ap_transformed_train = ap_transformed[:train_size]
ap_transformed_test =  ap_transformed[train_size:]

- ⑥ Auto ARIMA 적용 및 결과 확인
    - seasonal : False인 경우 non-seasonal model로 제한합니다.
    
    - suppress_warnings : ARIMA를 실행하면 생기는 에러문구를 나오지 않도록 해줍니다.
    
    - trace : True일 때 오류(error)가 발생하면 일부 내용을 출력합니다.
    
    - max_D : seasonal의 최대 값을 설정할 수 있습니다.

In [ ]:
# AutoARIMA
# seasonal = True, max_D(계절차분) = 12
model = pm.AutoARIMA(seasonal = True, suppress_warnings = True, trace = True, max_D = 12)
res = model.fit(ap_transformed_train)

In [ ]:
# confidence interval까지 도출
preds, conf_int = res.predict(n_periods=ap_transformed_test.shape[0], return_conf_int=True)

# 아래의 코드 결과값을 서로 비교 분석해봅시다.
print(preds.shape)
print(type(preds))
print('--'*40)
print(preds.shape[0])
print(type(preds.shape[0]))
print('--'*40)
print(preds)
print('--'*40)
print(conf_int)

In [ ]:
# 예측 성능을 시각화합니다.
# 0. train과 test data를 축(axis)으로 만들었습니다.
# 1. x축은 test으로 만들어 놓은 train data를 사용하고, y축에 train data를 사용하여 train data를 시각화 합니다.
# 2. 1번 그래프에서 정답(AutoARIMA를 통해 만든 예측 값)에 해당하는 일자형 그래프가 생성됩니다.
# 3. 원래 가지고 있던 데이터셋의 y값(지도 학습의 정답)을 산점도(scatter plot)로 생성합니다.
# 4. 2, 3번의 분포에 해당하는 구간을 채워 그래프로 나타냈습니다.
x_axis = np.arange(ap_transformed_train.shape[0] + preds.shape[0]) # 115 + 29, plot을 그리기 위해 x축을 생성합니다. # 0
plt.plot(x_axis[:ap_transformed_train.shape[0]],ap_transformed_train, alpha=0.75) # 1
plt.plot(x_axis[ap_transformed_train.shape[0]:], preds, alpha=0.75) # 2
plt.scatter(x_axis[ap_transformed_train.shape[0]:],ap_transformed_test,
            alpha=0.4, marker='o')  # 3 기존 test data는 scatter
plt.fill_between(x_axis[-preds.shape[0]:],
                 conf_int[:, 0], conf_int[:, 1],
                 alpha=0.1, color='b') # 4
plt.title("Log Transformed Air Passengers Forecast") # plot의 제목
plt.show() # 4개의 plot을 시각화 합니다.

- ⑦ ARCH 모델 실습해보기

In [ ]:
# ⑦-1 : 데이터 및 라이브러리 불러오기
import datetime as dt # 날짜와 시간을 조작할 수 있는 Python 라이브러리 입니다 .
import arch.data.sp500 # S&P 500 데이터셋을 arch에서 불러옵니다.

# ⑦-2 : 필요한 데이터로 가공하고 시각화를 통해 확인하기
st = dt.datetime(2002, 1, 1) # start date
en = dt.datetime(2022, 1, 1) # end date
data = arch.data.sp500.load() # s&p 500 데이터 불러오기
market = data["Adj Close"] # 종가만 활용
returns = 100 * market.pct_change().dropna() # return값으로 변환
ax = returns.plot() # 수익률을 그래프로 만들기
xlim = ax.set_xlim(returns.index.min(), returns.index.max()) # xlim은 x축 값의 보기를 제한 합니다.
plt.show()

In [ ]:
# ⑦-3 : ARCH 모델 적용
from arch import arch_model

am = arch_model(returns) # return을 arch모델에 적합
res = am.fit(update_freq=5)

# ⑦-4 : 지표를 통한 결과 확인
print(res.summary())
# 다양한 결과가 제시되어있으나, 수업때 배웠던 p-value, AIC, BIC위주로 내용을 파악
# 아래에 있는 volatility model을 보면 alpha[1], Beta[1]은 GARCH(1,1)을 의미하며, p-value를 통해 신뢰도 95%에서 이들이 유의미함을 도출

In [ ]:
# ⑦-5 : 시각화를 통한 결과값 확인
res.plot()